In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Bidirectional
from keras.layers import SimpleRNN, Dense, LSTM
import numpy as np


2025-05-23 09:33:31.983979: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print(tf.__version__)

2.17.0


In [3]:

# Define parameters (same as used during training)
max_features = 20000
maxlen = 80
embedding_dim = 128
lstm_out = 128


In [4]:
# Load the IMDB dataset and preprocess, and get word indexing.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
word_index = imdb.get_word_index()


In [5]:

# Build the BiLSTM model
model = keras.Sequential()
model.add(layers.Embedding(max_features, embedding_dim))
model.add(Bidirectional(LSTM(lstm_out)))
model.add(Dense(1, activation='sigmoid')) # Output layer for binary classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [6]:
type(x_train), type(y_train)

(numpy.ndarray, numpy.ndarray)

In [7]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)


(25000, 80) (25000,)
(25000, 80) (25000,)


In [8]:

# Train the model
# model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=128)



In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2025-05-23 09:34:33.365178: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15788566141759408503
 xla_global_id: -1]

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=1, batch_size=32)

In [ ]:
# Evaluate the model
score, acc = model.evaluate(x_test, y_test, verbose=0)
print('Test score:', score)
print('Test accuracy:', acc)

In [11]:
!python --version

Python 3.9.20


In [ ]:

def predict_sentiment(review_text):
    """Predicts the sentiment of a given movie review."""
    # Preprocess the review text
    review_tokens = review_text.lower().split()  # Tokenize and lowercase
    review_indices = [word_index.get(token, 0) for token in review_tokens] # Convert words to indices

    # Filter out unknown tokens (index 0 is reserved for out-of-vocabulary words)
    review_indices = [index for index in review_indices if index != 0]

    padded_review = sequence.pad_sequences([review_indices], maxlen=maxlen)

    # Make prediction
    prediction = model.predict(padded_review)

    # Interpret the prediction (sigmoid output)
    if prediction[0] > 0.5:
        sentiment = "Positive"
    else:
        sentiment = "Negative"
    return sentiment, prediction[0]


# Example usage
new_review1 = "This movie was absolutely fantastic! The acting was superb."
sentiment1, score1 = predict_sentiment(new_review1)
print(f"Review: {new_review1}")
print(f"Sentiment: {sentiment1}, Score: {score1}")

new_review2 = "What a terrible movie! I hated every minute of it."
sentiment2, score2 = predict_sentiment(new_review2)
print(f"Review: {new_review2}")
print(f"Sentiment: {sentiment2}, Score: {score2}")

new_review3 = "The movie was okay. Not great, not terrible."
sentiment3, score3 = predict_sentiment(new_review3)
print(f"Review: {new_review3}")
print(f"Sentiment: {sentiment3}, Score: {score3}")
